In [1]:
from bn256 import *
import random
from copy import copy
import time

In [2]:
def timer(func):
    def _func(*args, **kwargs):
        t = time.process_time()
        res = func(*args, **kwargs)
        t = time.process_time() - t
        print('{0}: time elapsed {1:.8f} s.' .format(func.__name__, t))
        return res

    return _func

In [3]:
def hamminge_distance(s1, s2):
    l = len(s1)
    assert l == len(s2), "Undefined for sequences of unequal length."
    return sum(el1 != el2 for el1, el2 in zip(s1, s2)) / l

In [4]:
a = (-1, 1)
def gen_vector(size):
    return tuple(random.choice(a) for i in range(size))

In [5]:
v1, v2 = gen_vector(16), gen_vector(16)

In [6]:
print(v1, v2, sep="\n")

(-1, -1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1)
(-1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, -1, 1, 1)


In [7]:
def inverse_G1(curve):
    inv = copy(curve)
    inv.y = inv.y.additive_inverse()
    return inv

In [8]:
def inverse_G2(twist):
    inv = copy(twist)
    inv.negate()
    return inv

In [9]:
def transfer_vec_to_G1(vec):
    return tuple(inverse_G1(curve_G) if p == -1 else curve_G for p in vec)

In [10]:
def transfer_vec_to_G2(vec):
    return tuple(inverse_G2(twist_G) if p == -1 else twist_G for p in vec)

In [11]:
def gen_keys(size):
    return tuple(random.randrange(2, order) for i in range(size))

In [12]:
v1points = transfer_vec_to_G2(v1)
v2points = transfer_vec_to_G1(v2)

In [13]:
v1points

(((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (20666913350058776956210519119118544732556678129809273996262322366050359951122,17778617556404439934652658462602675281523610326338642107814333856843981424549)),
 ((2116796163654258025501177006657054

In [14]:
v2points

((1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781))

In [15]:
h1, h2 = random.randrange(2, order), random.randrange(2, order)
H1, H2 = curve_G.scalar_mul(h1), twist_G.scalar_mul(h2)

In [16]:
h1, h2

(382324233672638109700936912166257118467845530742063560796613824519336616355,
 54130793962108488460732269599280222781302178832974372764195107787595715294227)

In [17]:
@timer
def master_key_generation():
    s, t = gen_keys(16), gen_keys(16)
    u, v = gen_keys(18), gen_keys(18)
    gen1_h = tuple(point_add(curve_G.scalar_mul(si), H1.scalar_mul(ti)) for si, ti in zip(s, t))
    gen2_h = tuple(point_add(twist_G.scalar_mul(ui), H2.scalar_mul(vi)) for ui, vi in zip(u, v))
    return gen1_h, s, t, gen2_h, u, v

In [18]:
gen1_h, s, t, gen2_h, u, v = master_key_generation()

master_key_generation: time elapsed 9.95312500 s.


In [19]:
gen1_h, s, t, gen2_h, u, v

(((2290753780687387217697920175288757367987763742362565614065639000862977745808, 36602278553715861457164510045908132859864824144848343436187632060831193974084),
  (19316298341081485010193421857530065184147326844099545614437284114181851725575, 39959840839035082352683352346731678503853892127255888386320082600436354127038),
  (57359197593646851401402282684109884709319747481889255180493094679714532938330, 5440757309174806428892297924015673533162224576327768477805536888503386676014),
  (48398540579787569011200165725928597886192672665824047983632090906555462667285, 51935985524512256259192049182273674101338841851469284741978989858050584108773),
  (21479047270019529480155882768360386912544236221981347554956081058042314211226, 14575862775387119220083256691556778859232789869606662228786735578379989725784),
  (527464782578499882387678739085121385168419993107257562317540711243552765447, 9963760947919841595440056448867943824797541205836113009223810264261632767620),
  (533404477024804148262575881612

In [20]:
@timer
def decryption_key_generation(gen2_h, s, t, v1points):
    n = len(v1points)
    
    r0 = random.randrange(2, order)
    reg_template = [twist_G.scalar_mul(r0), H2.scalar_mul(r0)]
    
    d03 = v1points[0].scalar_mul(s[0])
    d04 = v1points[0].scalar_mul(t[0])
    for i in range(1, n):
        d03 = point_add(d03, v1points[i].scalar_mul(s[i]))
        d04 = point_add(d04, v1points[i].scalar_mul(t[i]))
    reg_template.append(point_add(inverse_G2(d03), gen2_h[0].scalar_mul(r0)))
    reg_template.append(point_add(inverse_G2(d04), gen2_h[1].scalar_mul(r0)))

    for i in range(n):
        reg_template.append(point_add(v1points[i], gen2_h[i + 2].scalar_mul(r0)))
    
    return tuple(reg_template)

In [21]:
DK = decryption_key_generation(gen2_h, s, t, v1points)

decryption_key_generation: time elapsed 12.14062500 s.


In [22]:
len(DK)

20

In [23]:
DK

(((3661013084416119144577761122124610575184494527278789125473899244284947144894,53493052781177641584704770975351329277768647966741144175787548858868491062841), (42621804673319073319089256202490818404379015204008649070337245406098688244187,22205144021593428692558948002035880006208470641444533490774567310219923719265)),
 ((38163975926742977361712604925837344271671396271730420478117649913294209693641,11252246541808682876863540045073763492785086375952038834105718698159780781093), (6762728623249830343142096906816146486784217176233058237152686563245550310705,45658830275682490813634868536808613919957835367672804603486739543571827295357)),
 ((28018776230661763928364561832355284581898090279048781734085635633902080015735,40378109811707012741457425890691078053642181102819499888489914297139078241613), (35142283809549930668363090997146894766767872488986327677243777927052541049952,45022292833386661462858441376491554385885455186914116683626345207681670421791)),
 ((432966450821059337999327623568981649

In [24]:
@timer
def encryption_authentication(gen1_h, u, v, v2points):
    n = len(v2points)
    
    r0 = random.randrange(2, order)
    auth_template = [curve_G.scalar_mul(0), curve_G.scalar_mul(0), curve_G.scalar_mul(r0), H1.scalar_mul(r0)]
    
    for ci, h in zip(v2points, gen1_h):
        auth_template.append(point_add(ci, h.scalar_mul(r0)))
        
    c01 = auth_template[2].scalar_mul(u[0])
    c02 = auth_template[2].scalar_mul(v[0])
    for i in range(1, n + 2):
        c01 = point_add(c01, auth_template[i + 2].scalar_mul(u[i]))
        c02 = point_add(c02, auth_template[i + 2].scalar_mul(v[i]))
    
    auth_template[0] = point_add(auth_template[0], inverse_G1(c01))
    auth_template[1] = point_add(auth_template[1], inverse_G1(c02))
    
    return tuple(auth_template)

In [25]:
CT = encryption_authentication(gen1_h, u, v, v2points)

encryption_authentication: time elapsed 3.37500000 s.


In [26]:
len(CT)

20

In [27]:
CT

((49033905566623438713720655274319886616147964879738069937231821016614610529350, 51617054631488219792905561373819156912232391852581156045014863182902485183502),
 (61843880327229135894596988281586803668727361862004163951859360556418515411127, 27171389878538148778173763737241094074270590126988238255449940896278771997332),
 (52898764468801512345901561991493763492593939141628678181595649203357888723043, 13191265152426561569405625675416662656584105012181898403489963806508864959247),
 (20911511170166867896724246190149757724385131296696286054794738365692879980545, 33733891808143578723953117051835568949399810900405859133638445511358326764479),
 (32202912332486626850298537092288562478312723125909664036370223379584470017449, 10400895531089581219381308413483546711674175122351016565452369424975189737168),
 (22384533967318985022310432032592723349022609036542992822219958619014191325241, 7989858546629853166386952077984781270475946449900371756533636190369155707902),
 (426503994746605566297369549657278

In [28]:
gt = optimal_ate(twist_G, curve_G)
logarithmic_table = [gt.exp(i) for i in range(16, -1, -2)]
gtm = optimal_ate(twist_G, inverse_G1(curve_G))
logarithmic_table.extend(gtm.exp(i) for i in range(2, 17, 2))

In [29]:
for i, el in enumerate(logarithmic_table):
    print(i, ": ", el)

0 :  (((15429308596100125345042025182838740585933899260990097211943232471185708209892,21036769640615344552925704451163498369215862996620789540674973406371475898801),(34643419400173764267828846220737244479195263121357284576095582127626220190690,39217953906512156784965537049391267021797863115575109425638059426537342418239),(29362508885087302295248031767237294697462853748363034881398247042502825752042,33065606296478674290051087119664030123566328954069047137872363079287669349803)),((44150688783494038190860128993786071429009665932812059212008957591291015396555,25855485444396824150330723346354416914393823131049522641950650987664836921571),(14355433543029713763013415741005074744216778912056694539583678910055649667776,46280493583452157638432486289407345007584489845082308921018418732775447515998),(35438949274284069241468591891906709017152328378869050685915198888251448805767,45559280046590109824720435394540600218774466880098159450808971756061223256757)))
1 :  (((597783055027885551703832584052683

In [30]:
@timer
def decryption_authentication(DK, CT):
    d = optimal_ate(DK[0], CT[0])
    for i in range(1, 20):
        d = d.mul(optimal_ate(DK[i], CT[i]))
    return logarithmic_table.index(d)

In [31]:
a = decryption_authentication(DK, CT)

decryption_authentication: time elapsed 12.93750000 s.


In [32]:
a

5

In [33]:
a / 16

0.3125

In [34]:
hamminge_distance(v1, v2)

0.3125